# Problem Definition for RFM Analysis
📌 FLO, an online shoe store, wants to segment its customers and determine marketing strategies according to these segments. For this, the behaviors of the customers will be defined and groups will be formed according to the clusters in these behaviors.

## Import necessary libraries

In [1]:
!pip install lifetimes
import numpy as pd
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)
pd.set_option("display.float_format", lambda x: '%.4f' % x)
flo_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/flo_data_20k.csv")
df = flo_data.copy()
df.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 584 kB 29.9 MB/s 


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]


## General Information About Dataset

In [2]:
def check_df(dataframe,head=10):
  print("################################ Head ################################\n")
  print(dataframe.head(head))
  print("################################ Tail ################################\n")
  print(dataframe.tail(head))
  print("################################ Shape ################################\n")
  print(dataframe.shape)
  print("################################ Types ################################\n")
  print(dataframe.dtypes)
  print("################################ NA ################################\n")
  print(dataframe.isnull().sum())
  print("################################ Qurtiles ################################\n")
  print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

################################ Head ################################

                              master_id order_channel last_order_channel first_order_date last_order_date last_order_date_online last_order_date_offline  order_num_total_ever_online  order_num_total_ever_offline  customer_value_total_ever_offline  customer_value_total_ever_online       interested_in_categories_12
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline       2020-10-30      2021-02-26             2021-02-21              2021-02-26                       4.0000                        1.0000                           139.9900                          799.3800                           [KADIN]
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile       2017-02-08      2021-02-16             2021-02-16              2020-01-10                      19.0000                        2.0000                           159.9700                         1853.5800  [ERKEK, COCUK, K

## Create RFM

In [3]:
def create_rfm(dataframe,csv=False):
  # Data Preprocessing
  dataframe["order_num_total"] = dataframe["order_num_total_ever_offline"] + dataframe["order_num_total_ever_online"]
  dataframe["customer_value_total_price"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
  for col in dataframe.columns:
    if "date" in col:
      dataframe[col] = dataframe[col].apply(pd.to_datetime)
  
  # Calculating RFM Metrics
  today_date = dt.datetime(2021,6,1)
  rfm = dataframe.groupby(["master_id"]).agg({'master_id': lambda master_id: master_id,
                                              'last_order_date': lambda last_order_date: (today_date - last_order_date.max()).days,
                                              'order_num_total': lambda order_num_total: order_num_total,
                                              'customer_value_total_price': lambda customer_value_total_price: customer_value_total_price.sum()})
  rfm.columns = ["master_id","recency","frequency","monetary"]
  rfm = rfm[rfm["monetary"] > 0]

  # Calculating RFM Scores
  rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])
  rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
  rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])
  rfm["RFM_SCORE"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)

  # Set RFM Names
  seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_to_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loaylists',
    r'5[4-5]': 'champions'
  }
  rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map,regex=True)

  # Save Results to CSV File
  if csv:
    rfm.to_csv("rfm.csv")
  
  return rfm

In [4]:
rfm = create_rfm(df,csv=True)

In [5]:
rfm[["segment","recency","frequency","monetary"]].groupby(["segment"]).agg(['mean','count'])

recency       frequency        monetary      
                        mean count      mean count      mean count
segment                                                           
about_to_sleep      114.0316  1643    2.4066  1643  361.6494  1643
at_risk             242.3290  3152    4.4702  3152  648.3250  3152
cant_loose          235.1591  1194   10.7169  1194 1481.6524  1194
champions            17.1422  1920    8.9651  1920 1410.7089  1920
hibernating         247.4263  3589    2.3915  3589  362.5833  3589
loyal_customers      82.5579  3375    8.3564  3375 1216.2572  3375
need_to_attention   113.0372   806    3.7395   806  553.4366   806
new_customers        17.9762   673    2.0000   673  344.0495   673
potential_loaylists  36.8697  2925    3.3108  2925  533.7413  2925
promising            58.6946   668    2.0000   668  334.1533   668

**Q1** : FLO includes a new women's shoe brand. The product prices of the brand it includes are above the general customer preferences. For this reason, it is desired to contact the customers in the profile that will be interested in the promotion of the brand and product sales. Those who shop from their loyal customers (champions, loyal_customers) and women category are the customers to be contacted specifically. Save the id numbers of these customers to the csv file.

In [6]:
target_segment_customer_id = rfm[rfm["segment"].isin(["champions","loyal_customers"])]["master_id"]
customer_id = df[(df["master_id"].isin(target_segment_customer_id)) & (df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]
customer_id.to_csv("new_womens_shoe_brand.csv")

**Q2** : Nearly 40% discount is planned for Men's and Children's products. It is aimed to specifically target customers who are good customers in the past, but who have not shopped for a long time, who are interested in the categories related to this discount, who should not be lost, those who are asleep and new customers. Save the ids of the customers in the appropriate profile to the csv file.

In [7]:
target_segment_customer_id = rfm[rfm["segment"].isin(["cant_loose","hibernating","new_customers"])]["master_id"]
customer_id = df[(df["master_id"].isin(target_segment_customer_id)) & ((df["interested_in_categories_12"].str.contains("ERKEK")) | (df["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]
customer_id.to_csv("discount_target_customer_id.csv")

# Problem Definition for CLTV Prediction
📌 FLO wants to set a roadmap for sales and marketing activities. In order for the company to make a medium-long-term plan, it is necessary to estimate the potential value that existing customers will provide to the company in the future.

## Find outlier and then replace with thresholds

In [8]:
def outlier_thresholds(dataframe,variable):
  quartile1 = dataframe[variable].quantile(0.01)
  quartile3 = dataframe[variable].quantile(0.99)
  interquartile_range = quartile3 - quartile1
  up_limit = quartile3 + 1.5 * interquartile_range
  low_limit = quartile1 - 1.5 * interquartile_range
  return low_limit, up_limit

def replace_with_thresholds(dataframe,variable):
  low_limit, up_limit = outlier_thresholds(dataframe,variable)
  dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
  dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit, 0)

## Create CLTV Prediction

In [9]:
def create_cltv_prediction(dataframe,month=3,csv=False):
  # Data Preprocessing
  variable_list = ["order_num_total_ever_online",
          "order_num_total_ever_offline",
          "customer_value_total_ever_offline",
          "customer_value_total_ever_online"]
  for col in variable_list:
    replace_with_thresholds(dataframe,col)
  
  dataframe.dropna(inplace=True)
  dataframe["order_num_total"] = dataframe["order_num_total_ever_offline"] + dataframe["order_num_total_ever_online"]
  dataframe["customer_value_total_price"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]

  for col in dataframe.columns:
    if "date" in col:
      dataframe[col] = dataframe[col].apply(pd.to_datetime)
  
  #Preparation of lifetime data structure
  today_date = dt.datetime(2021, 6, 1)
  cltv_df = dataframe[["master_id","last_order_date","first_order_date","order_num_total","customer_value_total_price"]]
  cltv_df.rename(columns={"master_id": "customer_id"}, inplace=True)
  cltv_df["recency_cltv_weekly"] =  (cltv_df["last_order_date"] - cltv_df["first_order_date"]).dt.days
  cltv_df["T_weekly"] = (today_date - cltv_df["first_order_date"]).dt.days
  cltv_df["frequency"] = cltv_df["order_num_total"]
  cltv_df["monetary_cltv_avg"] = cltv_df["customer_value_total_price"] / cltv_df["frequency"]
  cltv_df = cltv_df.loc[:, ["customer_id","recency_cltv_weekly","T_weekly","frequency","monetary_cltv_avg"]]
  cltv_df = cltv_df[cltv_df["frequency"] > 1]
  cltv_df["recency_cltv_weekly"] = cltv_df["recency_cltv_weekly"] / 7
  cltv_df["T_weekly"] = cltv_df["T_weekly"] / 7
  cltv_df.index = cltv_df["customer_id"]
  del cltv_df["customer_id"]
  
  # Set the BG-NBD Model
  bgf = BetaGeoFitter(penalizer_coef=0.001)
  bgf.fit(cltv_df["frequency"],cltv_df["recency_cltv_weekly"],cltv_df["T_weekly"])
  ## Estimate expected purchases from customers in 3 months and add exp_sales_3_month to cltv dataframe
  cltv_df["exp_sales_3_month"] = bgf.predict(4*3,
                                           cltv_df["frequency"],
                                           cltv_df["recency_cltv_weekly"],
                                           cltv_df["T_weekly"])
  ## 6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv dataframe'ine ekleyiniz.
  cltv_df["exp_sales_6_month"] = bgf.predict(4*6,
                                           cltv_df["frequency"],
                                           cltv_df["recency_cltv_weekly"],
                                           cltv_df["T_weekly"])

  # Set the Gamma-Gamma Model
  ggf = GammaGammaFitter(penalizer_coef=0.01)
  ggf.fit(cltv_df["frequency"],cltv_df["monetary_cltv_avg"])

  ## Estimate the average value of the customers and add it to the cltv dataframe as exp_average_value.
  cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],cltv_df["monetary_cltv_avg"])

  # Calculating cltv with BG-NBD Model and Gamma-Gamma Model
  cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency_cltv_weekly"],
                                   cltv_df["T_weekly"],
                                   cltv_df["monetary_cltv_avg"],
                                   time=6, # month
                                   freq="W",
                                   discount_rate=0.01)
  cltv = cltv.reset_index()
  cltv_final = cltv_df.merge(cltv, on="customer_id", how="left")
  
  # Creation of Segment
  cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D","C","B","A"])
  
  # Export All Results to CSV File
  if csv:
    cltv_final.to_csv("cltv_prediction.csv")
  
  return cltv_final

In [10]:
create_cltv_prediction(df)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,clv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,193.6327,395.7332,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.6650,199.4307,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,120.9676,170.2242,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,67.3201,98.9455,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,114.3251,95.0117,D
...,...,...,...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,41.1429,88.4286,3.0000,133.9867,0.4858,0.9716,141.3604,144.1017,C
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,42.2857,65.2857,2.0000,195.2350,0.4804,0.9609,210.7224,212.4408,B
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,88.7143,89.8571,3.0000,210.9800,0.4816,0.9632,221.7752,224.1308,B
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,98.4286,113.8571,6.0000,168.2950,0.6102,1.2204,172.6475,221.0789,B


In [12]:
cltv_final = create_cltv_prediction(df)
cltv_final.groupby(["segment"]).agg({'clv': ["mean","min","max"]})

clv                   
            mean      min       max
segment                            
D        80.3405  12.1080  112.2454
C       138.3115 112.2484  165.4689
B       199.5328 165.4692  240.0579
A       362.3155 240.0854 3327.7770